In [1]:
import pandas as pd
import numpy as np
import math
#from IPython.display import display

# Made by Henric Pietro Vicente Gil please credit me in your work :)

In [2]:
# Cria a variável que contem a lista de arqquivos que voce quer olhar, se for só um deixa como string

# .\Clinical Data\Re-Filter.xlsx
# Toxic Hepatitis
# ".\Clinical Data\Filtered_Clinical_Data.xlsx"
# "Filtered Toxic Hepatitis"

file="..\..\Clinical Data\Re-Filter.xlsx"
#sheet="Filtered Toxic Hepatitis"

In [3]:
def isInt(i):
    """Check if it is an integer"""
    try:
        int(i)
        return True
    except ValueError:
        return False

def isFloat(f):
    try:
        float(f)
        return True
    except ValueError:
        return False

def convertTo_mg(df,column="Dose"):
    """Pass a series like (aka: df.column) of measurement strings, convert the values into miligram numbers, returns a list o lists with a number and the unit"""   
    
    micro="\u00B5"
    measure=[]
    
    for dose in df.loc[:,column]:
        # num=[n for n in number if isInt(n) or n=="."]
        num=""
        unit=""
        nums=[]

        # Ensures it is not nan or None
        if type(dose) is str:
            
            for i,n in enumerate(dose):
                # Avoids unprecise entries
                if n in ["<",">"]:
                    continue

                if isInt(n) or n=="." and num.find(".")==-1:
                    num+=n

                if isInt(n)==False:
                    unit+=n

                if n in ["," ,"-", "\\", "/"] and num !="":
                    num.strip(" ,-\\/")
                    nums.append(float(num))
                    num=""
                if len(nums)>1 and "" not in nums:
                    avg=np.mean(nums)

                elif len(nums)==0 and num!="": 
                    avg=float(num)
            # unit=[str(s) for s in number if isInt(s)==False]
            measure.append([avg, str(unit).strip(" -.")])
    
    for i,m in enumerate(measure):
        #print(m)
        u=str(m[1])
        n=m[0]

        # if you want to use the strings that contain unit values uncomment the commented lines
        if u.find("kg")!=-1 and u.find("mg")==-1:
            measure[i][0]=n*1_000_000
            #measure[i][1]=str(measure[i][1]).replace("kg","mg")
            
        #might be with special char
        elif u.find(f"{micro}g")!=-1 or unit.find("ug")!=-1:
            measure[i][0]=n/1000
            #measure[i][1]=str(measure[i][1]).replace("ug","mg")

        elif u.find("cg")!=-1:
            measure[i][0]=n*10
            #measure[i][1]=str(measure[i][1]).replace("cg","mg")
        
        if u.find("g")==0:
            measure[i][0]=n*1000
            #measure[i][1]=str(measure[i][1]).replace("g","mg")
    
    solved_col=pd.Series(measure)
        
    return solved_col
#Method that calculates the standard deviation with a number list and the mean, but use np.std() instead
def stdCalculation(numList,mean):
    n=len(numList)+1
    soma=0
    for x in numList:
        soma+=(x-mean)**2
    std=math.sqrt((soma)/n)
    return std
#Method to calculate the z score for each number in a array of numbers, returns a list of z-scores of each number relevant to the array
def z_scorer(nums: list,mean: float or int,std: float or int):
    
    z_scores=[]
    for x in nums:
        z=(x-mean)/std
        z_scores.append(z)
    return z_scores

# Most of the action happens here
def idOutliers(df: pd.DataFrame,name_col: str="Drug",value_col: str="Dose",max_z: float=1.8):
    dict_rows={}
    means={}
    df=df.dropna()
    
    values=[num for num in df.loc[:,value_col]]
    names=[drugs for drugs in df.loc[:,name_col]]
    
    for i,name in enumerate(names):
        
        if name not in dict_rows:
            dict_rows[name]=[]
            
            dict_rows[name].append(values[i])            
        else: 
            dict_rows[name].append(values[i])
    
    for n in dict_rows:
        media=[]
        if len(dict_rows[n])>1:
            means[n]=np.mean(dict_rows[n])
            
            if means[n]/dict_rows[n][0]!=1:
                std=np.std(dict_rows[n])
            else: 
                std=1
            
            z= z_scorer(dict_rows[n],means[n],std)
            
            for _i,i in enumerate(z):
                if abs(i)<=max_z:
                    media.append(dict_rows[n][_i])
            
            med=np.mean(media)
            # atrubutes where in the column specified is equal to the name of the current row, and replaces the value (dose) of that row 
            df.loc[df[name_col]==n, value_col]=med
        else:
            df.loc[df[name_col]==n, value_col]=dict_rows[n]
    return df
#computes in the order necessary to generate the dataframes 
def organize(file,sheet):
    #read
    df=pd.read_excel(file,sheet_name=sheet)
    #convert
    new_doses=convertTo_mg(df)
    #assign to the df
    df["Dose"]=new_doses
    #generate
    z=idOutliers(df)
    z=z.drop_duplicates()
    z.to_excel(f"{sheet}.xlsx")
    #confirm
    print(f"generated {sheet}.xlsx dataframe")

# Made to work with a single file and a list of files
def main(files):
    if type(files) is str:
        sheets=pd.ExcelFile(files).sheet_names
        for sheet in sheets:
            organize(files,sheet)
    for file in files:
        sheets=pd.ExcelFile(file).sheet_names
        for sheet in sheets:
            organize(file,sheet)


In [4]:
#Execute everything
main(file)

D:\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
D:\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


generated Cholestasis.xlsx dataframe
generated Liver Injury.xlsx dataframe
generated Cholestatic Jaundice.xlsx dataframe
generated Jaundice.xlsx dataframe
generated Hyperbilirrubinemia.xlsx dataframe
generated Hepatomegaly.xlsx dataframe
generated Hepatocellular injury.xlsx dataframe
generated Hepatobiliary disorder.xlsx dataframe
generated Hepatic Necrosis.xlsx dataframe
generated Abnormal Hepatic Function.xlsx dataframe
generated Hepatic Failure.xlsx dataframe
generated Cholelithiasis.xlsx dataframe
generated Cholecystitis.xlsx dataframe
generated Autoimune hepatits.xlsx dataframe
generated Hepatic Cirrhosis.xlsx dataframe
generated Hepatitis.xlsx dataframe
generated Acute Hepatitis.xlsx dataframe
generated Cholestatic Hepatitis.xlsx dataframe
generated Fulminant Hepatitis.xlsx dataframe
generated Toxic Hepatitis.xlsx dataframe
generated Hepatic Steatosis.xlsx dataframe


PermissionError: [Errno 13] Permission denied: '.'